In [1]:
import pandas as pd

In [12]:
scoreMatrix = [[] for _ in range(200)]
with open('labels_200_1000.txt', 'r') as file:
    for line in file.readlines():
        line = line.replace('\n', '')
        [nurse2Patient, score] = line.split(': ')

        [nurse, patient] = nurse2Patient.split('-')
        score = float(score)

        nurseIndex = int(nurse)
        patientIndex = int(patient)

        scoreMatrix[nurseIndex].append(score)

In [13]:
with open('inductive/train/labels.txt', 'w') as file:
    for i in range(160):
        for j in range(800):
            file.write(f'{i}-{j}: {scoreMatrix[i][j]}\n')

In [14]:
with open('inductive/val/labels.txt', 'w') as file:
    for i in range(160, 180):
        for j in range(800, 900):
            file.write(f'{i - 160}-{j - 800}: {scoreMatrix[i][j]}\n')

In [15]:
with open('inductive/test/labels.txt', 'w') as file:
    for i in range(180, 200):
        for j in range(900, 1000):
            file.write(f'{i - 180}-{j - 900}: {scoreMatrix[i][j]}\n')

In [17]:
nurseData = pd.read_csv('nurseData_200.csv')
patientData = pd.read_csv('patientData_1000.csv')

nurseData[:160].to_csv('inductive/train/nurseData.csv', index=False)
nurseData[160:180].to_csv('inductive/val/nurseData.csv', index=False)
nurseData[180:].to_csv('inductive/test/nurseData.csv', index=False)

patientData[:800].to_csv('inductive/train/patientData.csv', index=False)
patientData[800:900].to_csv('inductive/val/patientData.csv', index=False)
patientData[900:].to_csv('inductive/test/patientData.csv', index=False)

In [3]:
k = 0.9
while k >= 0.5:
    with open(f'inductive/test/edge_nurse_{round(k, 1)}_dist.txt', 'w') as outputFile:
        edgeCount = 0
        with open(f'edge/200_1000/edge_nurse_{round(k, 1)}_dist.txt', 'r') as file:
            for line in file.readlines()[:-1]:
                line = line.replace('\n', '')
                [srcNode, tarNodes] = line.split(': ')

                srcNode = int(srcNode)
                if 180 <= srcNode < 200:
                    tarNodes = [node - 180 for node in map(int, tarNodes.split(', ')) if 180 <= node < 200]
                    if len(tarNodes) > 0:
                        edgeCount += len(tarNodes)
                        edgeInfo = ', '.join(map(str, tarNodes))
                        outputFile.write(f'{srcNode - 180}: {edgeInfo}\n')

        print(edgeCount)
        outputFile.write(f'edgeCount: {edgeCount}\n')

    k -= 0.1

20
30
60
106
190


In [6]:
k = 0.9
while k >= 0.5:
    with open(f'inductive/test/edge_patient_{round(k, 1)}_dist.txt', 'w') as outputFile:
        edgeCount = 0
        with open(f'edge/200_1000/edge_patient_{round(k, 1)}_dist.txt', 'r') as file:
            for line in file.readlines()[:-1]:
                line = line.replace('\n', '')
                [srcNode, tarNodes] = line.split(': ')

                srcNode = int(srcNode)
                if 900 <= srcNode < 1000:
                    tarNodes = [node - 900 for node in map(int, tarNodes.split(', ')) if 900 <= node < 1000]
                    if len(tarNodes) > 0:
                        edgeCount += len(tarNodes)
                        edgeInfo = ', '.join(map(str, tarNodes))
                        outputFile.write(f'{srcNode - 900}: {edgeInfo}\n')

        print(edgeCount)
        outputFile.write(f'edgeCount: {edgeCount}\n')

    k -= 0.1

104
252
676
1830
3800


In [5]:
embedMatrix = [[] for _ in range(200)]
with open('embeds_200_1000.txt', 'r') as file:
    for line in file.readlines():
        line = line.replace('\n', '')
        [pairs, embedInfo] = line.split(': ')

        [i, j] = map(int, pairs.split('-'))
        embedMatrix[i].append(embedInfo)

In [8]:
with open('inductive/test/embeds.txt', 'w') as file:
    for i in range(180, 200):
        for j in range(900, 1000):
            file.write(f'{i-180}-{j-900}: {embedMatrix[i][j]}\n')

In [39]:
m = {}
with open('inductive/val/edge_patient_0.9.txt', 'r') as file:
    lines = file.readlines()
    for line in lines[:-1]:
        line = line.replace('\n', '')
        [srcNode, tarNodes] = line.split(': ')

        srcNode = int(srcNode)
        tarNodes = list(map(int, tarNodes.split(', ')))

        m[srcNode] = tarNodes
    edgeCount = int(lines[-1].replace('\n', '').split(': ')[1])

In [31]:
nurseData = pd.read_csv('inductive/val/nurseData.csv')

edgeCount = 0
for srcIndex in m.keys():
    deleteList = []
    for tarIndex in m[srcIndex]:
        if nurseData.iloc[tarIndex]['在岗时间'] > nurseData.iloc[srcIndex]['在岗时间']:
            deleteList.append(tarIndex)

    for i in deleteList:
        m[srcIndex].remove(i)

    edgeCount += len(m[srcIndex])
edgeCount

8

In [40]:
with open('inductive/val/edge_patient_0.9_selfLoop.txt', 'w') as file:
    for i in range(100):
        if i not in m.keys():
            file.write(f'{i}: {i}\n')
            edgeCount += 1
        else:
            tarNodes = ', '.join(map(str, m[i]))
            file.write(f'{i}: {tarNodes}\n')

    file.write(f'edgeCount: {edgeCount}\n')

In [2]:
nurseData = pd.read_csv('nurseData.csv')[:200]
patientData = pd.read_csv('patientData.csv')[:1000]

nurseID_list = nurseData['护工ID'].tolist()
patientID_list = patientData['患者ID'].tolist()

In [3]:
m = {}
with open('护工_患者.txt', 'r') as file:
    for line in file.readlines():
        line = line.replace('\n', '')
        [nurseID, patients] = line.split('->')

        nurseID = int(nurseID)
        patients = eval(patients)

        m[nurseID] = patients

In [6]:
with open('n2p.txt', 'w') as file:
    for i, nurseID in enumerate(nurseID_list):
        tmp = []
        for patient in m[nurseID]:
            if patient in patientID_list:
                tmp.append(patientID_list.index(patient))

        if len(tmp) > 0:
            adjInfo = ' '.join(map(str, tmp))
            file.write(f'{i} {adjInfo}\n')